# Preparation of the two-step analysis

Author:<br>
Nora Poel (1,2)<br>
(1) CNRS/UGA/IRD/G-INP, IGE, Grenoble, France<br>
(2) University of Potsdam, Institute for Computer Science, Potsdam, Germany

This notebook serves to prepare the input data for the two-step analysis.

In [1]:
import numpy as np
import numpy.ma as ma
import numpy.linalg as linalg
import matplotlib.pyplot as plt
import xarray as xr
import re
from netCDF4 import Dataset
import os

import SWOTaugmentation as swota
import SWOTdenoise as swotd
import SWOTremovedoublenadir as swotn

%matplotlib inline

## 1. Process observation-ensemble

### 1.1 Remove double nadir in OBS0060
The SWOT simulator produces two nadir tracks in the middle of the track if passes are generated without the 20 km gap between the two swaths. They are identical and we need to remove one.

**Note**: ensemble need to be named as followed: <name><nnnn>.nc.bas. The module will extract the size of the ensemble from <nnnn>.

In [3]:
datapath = '/home/poeln/Work/obs_pass182'
inputensname = '/OBS0060.nc.bas'
ensname = datapath + inputensname

swotn.SWOTremovedoublenadir(ensname)
# results in: rmdc_OBS0060.nc.bas

Ensemble without double nadir in  /home/poeln/Work/obs_pass182/rmdc_OBS0060.nc.bas


### 1.2 Denoise OBS0060
This step is necessary to estimate the best parameters for $\mathrm{R}^+$ and therefore to generate $\mathrm{R}^+$ for the analysis. For explanation the [20180418-np-parametrization-R.ipynb](20180418-np-parametrization-R.ipynb).<br>

**Note**: To denoise an ensemble, the name of the ensemble cannot include a '.nc' (e.g. not 'path/ENS0060.nc.bas /vctgridSWOT0001.nc'). This is because function write_data in SWOTdenoise.py splits the name of the input file at the first .nc to create the name of the output file. Consequently, all denoised members of the ensemble get the same name and will be overwritten (f.e. path/ENS0060_denoised.nc instead of path/ENS0060.nc.bas/ vctgridSWOT0001.nc).

In [ ]:
# rename rmdc_OBS0060.nc.bas to rmdc_OBS0060

datapath = '/home/poeln/Work/obs_pass182'
inputensname = '/rmdc_OBS0060'
ensname = datapath + inputensname

for k in range(1,61):
    membername = (ensname+'/vctgridSWOT{:04d}.nc').format(k)
    swotd.SWOTdenoise(membername, method='gaussian', param=8)
    
# moved denoised files in directory rmdc_denoised_OBS0060.nc.bas 

### 1.3 Extend OBS0060
In this step we calculate first and second derivations along and across track for each member of the observation-ensemble (`OBS0060`) (calculation on `ssh_model`). For each derivation for each member a single netcdf file is created. 

In [ ]:
datapath = '/home/poeln/Work/obs_pass182'
inputensname = '/rmdc_OBS0060.nc.bas'
ensname = datapath + inputensname

swota.SWOTaugmentation(ensname, ens=True)

## 2. Process an observation
The processing of the observation depends on the way it was created: for an observation without gap follow 2.1 and 2.2; for an observation with gap follow 2.3.
### 2.1 Remove double nadir in observation
The SWOT simulator produces two nadir tracks in the middle of the track if passes are generated without 20 km gap between the two swaths. They are identical and we need to remove one.<br>
**Note**: The module SWOTremovedoublenadir was originally thought to be used only for ensembles as future SWOT data will have the 20 km gap between its two swaths and therefore do not need to be proceeded with the module. To remove a double nadir from a single observation we have to create an temporary ensemble (here: `obs_p182_SWOT0001.nc.bas`) consisting one member (the observation) named as followed: `vctgridSWOT0001.nc`. The result will be in `rmdc_<ensemblename>` (here: `rmdc_obs_p182_SWOT0001.nc.bas`).

In [3]:
datapath = '/home/poeln/Work/obs_pass182'
inputensname = '/obs_p182_SWOT0001.nc.bas'
ensname = datapath + inputensname

swotn.SWOTremovedoublenadir(ensname)
# result in: rmdc_obs_p182_SWOT0001.nc.bas

Ensemble without double nadir in  /home/poeln/Work/obs_pass182/rmdc_obs_p182_SWOT0001.nc.bas


### 2.2 Denoise an observation
We denoise the observation from 2.1. The input path for the denoising method cannot include a '.nc'. This is because function write_data in SWOTdenoise.py splits the name of the input file at the first .nc to create the name of the output file. We therefore need to rename the directory.<br>
Here, the folder is named `rmdc_obs_nogap` and, for clarity, the observation is renamed as `rmdc_obs_p182_SWOT0004.nc`.

In [4]:
datapath = '/home/poeln/Work/obs_pass182/rmdc_obs_nogap'
obsname = '/rmdc_obs_p182_SWOT0004.nc'
observation = datapath + obsname
swotd.SWOTdenoise(observation,method='gaussian',param=8)

# result in: rmdc_obs_nogap/rmdc_obs_p182_SWOT0004_denoised.nc

Filtered field in  /home/poeln/Work/obs_pass182/rmdc_obs_nogap/rmdc_obs_p182_SWOT0004_denoised.nc


### 2.3 Denoise and inpaint an observation
An observation with 20 km gap can be denoised and inpainted as followed:

In [ ]:
# denoise and inpaint an observation
datapath = '/home/poeln/Work/obs_pass182'
obsname = '/obs_p182_SWOT0004.nc'
observation = datapath + obsname
swotd.SWOTdenoise(observation,inpainting=True,method='gaussian',param=8)

### 2.4 Augment an observation
In this step we calculate first and second derivations along and across track for an observation. For each derivation  a single netcdf file is created. 

In [7]:
datapath = '/home/poeln/Work/obs_pass182/rmdc_obs_nogap'
inputensname = '/rmdc_obs_p182_SWOT0004_denoised.nc'
obsname = datapath + inputensname

swota.SWOTaugmentation(obsname, obs=True)

# result:
# rmdc_obs_p182_SWOT0004_denoised_D1a.nc
# rmdc_obs_p182_SWOT0004_denoised_D1c.nc
# rmdc_obs_p182_SWOT0004_denoised_D2a.nc
# rmdc_obs_p182_SWOT0004_denoised_D2c.nc
# rmdc_obs_p182_SWOT0004_denoised_SWOT.nc

Extended observation in  /home/poeln/Work/obs_pass182/rmdc_obs_nogap/
